# Thresholded Binary Image

### Importing libraries

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import os
from ipywidgets import interact, interactive
%matplotlib inline

### Load Camera Calibration Data created in [Camera Calibration](camera_calibration.ipynb)

In [2]:
camera_calibrate = pickle.load( open('./outputs/camera_calibration.p', 'rb' ))
mtx, dist = map(camera_calibrate.get, ('mtx','dist'))

fpath = './test_images/test3.jpg'
img_sample = cv2.imread(fpath)

## Use color transforms, gradients, etc., to create a thresholded binary image.

### Absolute Sobel Threshold

In [3]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x' :
        sobel = cv2.Sobel(gray,cv2.CV_64F,1,0)
    if orient == 'y':
        sobel = cv2.Sobel(gray,cv2.CV_64F,0,1)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    # 6) Return this mask as your binary_output image
    binary_output[(scaled_sobel>= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    #binary_output = np.copy(img) # Remove this line
    return binary_output

In [4]:
vals = (1, 255, 1)
orients = ['x','y']
@interact(orient=orients,
          thresh_min=vals,
          thresh_max=vals)
def g(orient='x',thresh_min=25,thresh_max=255):
    bin_img = abs_sobel_thresh( img_sample, orient, thresh_min, thresh_max)
    plt.gray()
    plt.imshow(bin_img)
    plt.show()

interactive(children=(Dropdown(description='orient', options=('x', 'y'), value='x'), IntSlider(value=25, descr…

### Magnitude Threshold

In [5]:
# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def magnitude_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output


In [6]:
vals = (1, 255, 1)
k_size = ( 1, 5, 2)
@interact(sobel_kernel=k_size,
          thresh_min=vals,
          thresh_max=vals)
def g(sobel_kernel=3,thresh_min=25,thresh_max=255):
    bin_img = magnitude_thresh( img_sample, sobel_kernel, (thresh_min, thresh_max))
    plt.gray()
    plt.imshow(bin_img)
    plt.show()
    return

interactive(children=(IntSlider(value=3, description='sobel_kernel', max=5, min=1, step=2), IntSlider(value=25…

### Direction Threshold

In [7]:
# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return the binary image
    return binary_output

In [8]:
vals = (0, np.pi/2, 0.01)
k_size = ( 1, 5, 2)
@interact(sobel_kernel=k_size,
          thresh_min=vals,
          thresh_max=vals)
def g(sobel_kernel=3,thresh_min=1.0,thresh_max=np.pi/2):
    bin_img = dir_threshold( img_sample, sobel_kernel, (thresh_min, thresh_max))
    plt.imshow(bin_img, cmap='gray')
    plt.show()
    return

interactive(children=(IntSlider(value=3, description='sobel_kernel', max=5, min=1, step=2), FloatSlider(value=…

### RGB Threshold

In [9]:
def rgb_threshold(img, thresh=(0, 255)):
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    
    bin_R = np.zeros_like(R)
    bin_R[ (R>thresh[0]) & (R<thresh[1])] = 1
    
    bin_G = np.zeros_like(G)
    bin_G[ (G>thresh[0]) & (G<thresh[1])] = 1
    
    bin_B = np.zeros_like(B)
    bin_B[ (B>thresh[0]) & (B<thresh[1])] = 1
    
    plt.subplots(1, 3, figsize=(20,10))
    
    plt.subplot(1, 3, 1)
    plt.title('Red', fontsize=30)
    plt.imshow(bin_R)
    plt.subplot(1, 3, 2)
    plt.title('Green', fontsize=30)
    plt.imshow(bin_G)

    plt.subplot(1, 3, 3)
    plt.title('Blue', fontsize=30)
    plt.imshow(bin_R)

    plt.show()

In [10]:
vals = ( 0, 255)
@interact(thresh_min=vals, thresh_max=vals)
def g(thresh_min=150,thresh_max=255):
    rgb_threshold( img_sample, (thresh_min, thresh_max))

interactive(children=(IntSlider(value=150, description='thresh_min', max=255), IntSlider(value=255, descriptio…

### HLS Threshold

In [11]:
def hls_threshold(img, thresh=(0, 255)):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,0]
    L = hls[:,:,1]
    S = hls[:,:,2]
    
    bin_H = np.zeros_like(H)
    bin_H[ (H>thresh[0]) & (H<thresh[1])] = 1
    
    bin_L = np.zeros_like(L)
    bin_L[ (L>thresh[0]) & (L<thresh[1])] = 1
    
    bin_S = np.zeros_like(S)
    bin_S[ (S>thresh[0]) & (S<thresh[1])] = 1
    
    plt.subplots(1, 3, figsize=(20,10))
    
    plt.subplot(1, 3, 1)
    plt.title('H_Channel', fontsize=30)
    plt.imshow(bin_H)
    plt.subplot(1, 3, 2)
    plt.title('L_Channel', fontsize=30)
    plt.imshow(bin_L)

    plt.subplot(1, 3, 3)
    plt.title('S_Channel', fontsize=30)
    plt.imshow(bin_S)

    plt.show()

In [12]:
vals = ( 0, 255)
@interact(thresh_min=vals, thresh_max=vals)
def g(thresh_min=150,thresh_max=255):
    hls_threshold( img_sample, (thresh_min, thresh_max))

interactive(children=(IntSlider(value=150, description='thresh_min', max=255), IntSlider(value=255, descriptio…

### Combine the two binary thresholds HLS and Sobel

In [13]:
def combile_thresholds(img, sobelx_thresh=(20,100), s_thresh=(170,255)):
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    sx_binaray = np.zeros_like(scaled_sobel)
    sx_binaray[(scaled_sobel>=sobelx_thresh[0])&(scaled_sobel<=sobelx_thresh[1])] = 1
    
    his = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = his[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    combined_binary = np.zeros_like(sx_binaray)
    combined_binary[(s_binary == 1) | (sx_binaray == 1)] = 1
    return combined_binary

In [14]:
vals = (0, 255)
@interact(sx_thresh_min=vals,
          sx_thresh_max=vals,
          s_thresh_min=vals,
          s_thresh_max=vals
         )
def g(sx_thresh_min=20,sx_thresh_max=100,s_thresh_min=170,s_thresh_max=255):
    bin_img = combile_thresholds( img_sample, (sx_thresh_min,sx_thresh_max), (s_thresh_min, s_thresh_max))
    plt.imshow(bin_img)
    plt.savefig('./outputs/image_combine_thresholds.png')
    plt.show()
    return

interactive(children=(IntSlider(value=20, description='sx_thresh_min', max=255), IntSlider(value=100, descript…